In [25]:
import torch
from torch import nn, optim
import torchvision
from torchvision import transforms, datasets, models, utils
from torch.utils.data import Dataset, DataLoader 
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.nn import functional as F
from skimage import io, transform
from torch.optim import lr_scheduler
from skimage.transform import AffineTransform, warp
from sklearn.metrics import roc_auc_score, mean_absolute_error

import pandas as pd
from pathlib import Path
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random
from efficientnet_pytorch import EfficientNet

INPUT_PATH = Path('../input')
TRAIN_PATH = INPUT_PATH / 'idao_dataset' / 'train'
PRIVATE_PATH = INPUT_PATH / 'idao_dataset' / 'private_test'
PUBLIC_PATH = INPUT_PATH / 'idao_dataset' / 'private_test'

RANDOM_SEED = 4444
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(RANDOM_SEED)

In [26]:
import glob
images = glob.glob(str(TRAIN_PATH / '**/*.png'), recursive=True)

train_images, test_images = train_test_split(images, shuffle = True, random_state = RANDOM_SEED)

In [27]:
def calc_metric(y_binary_true, y_binary_pred, y_reg_true, y_reg_pred):
    '''
    Competition metric
    '''
    
    roc = roc_auc_score(y_binary_true, y_binary_pred)
    mae = mean_absolute_error(y_reg_true, y_reg_pred)
    return 1000 * (roc - mae), roc, mae

In [140]:
mapper1 = {
    1:[1, 0, 0, 0, 0, 0],
    3:[0, 1, 0, 0, 0, 0],
    6:[0, 0, 1, 0, 0, 0],
    10:[0, 0, 0, 1, 0, 0],
    20:[0, 0, 0, 0, 1, 0],
    30:[0, 0, 0, 0, 0, 1],
}

mapper2 = {
    1:0,
    3:1,
    6:2,
    10:3,
    20:4,
    30:5,
}

reverse_mapping = {
    0: 1,
    1: 3,
    2: 6,
    3: 10,
    4: 20,
    5: 30
}

In [141]:
class DataGetter(Dataset):
    def __init__(self, image_paths, train=True, transform=None):
 
        self.image_paths = image_paths 
        self.transform=transform
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image=cv2.imread(self.image_paths[idx])
        if len(self.image_paths[idx].split('_')) == 18:
            particle_class = 1
            particle_energy = mapper2[int(self.image_paths[idx].split('_')[7])]
        else:
            particle_class = 0
            particle_energy = mapper2[int(self.image_paths[idx].split('_')[8])]

        sample={
            'image': np.uint8(image), 
            'particle_class': particle_class,
            'particle_energy': particle_energy
            }

        #Applying transformation
        if self.transform:
            sample['image']=self.transform(sample['image'])
            
        return sample

In [142]:
augs = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([128,128]),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), 
    ])

In [143]:
transformed_train_data = DataGetter(train_images, train=True, transform=augs)
transformed_test_data = DataGetter(test_images, train=False, transform=augs)

train_dataloader = DataLoader(transformed_train_data, batch_size=32, shuffle=True) #, num_workers=2
test_dataloader = DataLoader(transformed_test_data, batch_size=32, shuffle=True)

In [144]:
class CNN(nn.Module):
    def __init__(self, pretrained=False):
        super(CNN, self).__init__()
        if pretrained:
            self.model = EfficientNet.from_pretrained('efficientnet-b0')
        else:
            self.model = EfficientNet.from_pretrained('efficientnet-b0')
        
        self.fc0 = nn.Linear(1280, 64)
        self.fc1 = nn.Linear(64, 2)  # For classification
        self.fc2 = nn.Linear(64, 6)  # For another classification
        
    def forward(self, x):
        bs, _, _, _ = x.shape
        x = self.model.extract_features(x)
        x = F.adaptive_avg_pool2d(x, 1).reshape(bs, -1)
        x = self.fc0(x)
        particle_class = torch.softmax(self.fc1(x), dim = 1)
        particle_energy= torch.softmax(self.fc2(x), dim = 1)
        return {'particle_class': particle_class, 'particle_energy': particle_energy}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_CNN = CNN().to(device)

Loaded pretrained weights for efficientnet-b0


In [133]:
for d in train_dataloader:
    out = model_CNN(d['image'].to(device))
    break

In [145]:
#For binary output:particle_class
criterion_binary= nn.CrossEntropyLoss()
# For regression output:particle_energy
criterion_reg = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_CNN.parameters(), lr=1e-1)

In [152]:
def train_model(model, criterion_binary, criterion_reg, optimizer, n_epochs=25):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf
    for epoch in range(1, n_epochs):
        train_loss = 0.0
        valid_loss = 0.0
        comp_metric = 0 
        comp_val_metric = 0
        # train the model #
        model.train()
        for batch_idx, sample_batched in enumerate(train_dataloader):
            # importing data and moving to GPU
            

            image, particle_class, particle_energy = sample_batched['image'].to(device),\
                                             sample_batched['particle_class'].to(device),\
                                              sample_batched['particle_energy'].to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            output = model(image)
            label_class = output['particle_class']
            label_energy = output['particle_energy']
            
            # calculate loss
            particle_class = particle_class.squeeze().type(torch.LongTensor).to(device)
            particle_energy = particle_energy.squeeze().type(torch.LongTensor).to(device)
            
            print(particle_class, particle_energy)
            
            
            y_pred_binary = label_class.cpu().detach().numpy()[:, 1]
            y_true_binary = particle_class.cpu().detach().numpy()
            y_pred_reg = label_energy.cpu().detach().numpy()
            y_true_reg = particle_energy.cpu().detach().numpy()
            
            y_pred_reg = [reverse_mapping[x] for x in list(np.argmax(y_pred_reg, axis = 1))]
            y_true_reg = [reverse_mapping[x] for x in list(y_true_reg)]
            comp_metric, roc, mae = calc_metric(y_true_binary, y_pred_binary, y_true_reg, y_pred_reg)
            
            loss_binary = criterion_binary(label_class, particle_class)
            loss_reg = criterion_reg(label_energy, particle_energy)
            
            torch.cuda.empty_cache()
            loss = loss_binary + loss_reg
            # back prop
            loss.backward()
            # grad
            optimizer.step()
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            if batch_idx % 1 == 0:
                print('Epoch %d, Batch %d loss: %.6f Metric %.6f ROC AUC %.6f MAE %.6f' %
                  (epoch, batch_idx + 1, train_loss, comp_metric, roc, mae))
        # validate the model #
        model.eval()
        for batch_idx, sample_batched in enumerate(test_dataloader):
            image, particle_class, particle_energy = sample_batched['image'].to(device),\
                                             sample_batched['particle_class'].to(device),\
                                              sample_batched['particle_energy'].to(device)
            
            output = model(image)

            label_class = output['particle_class']
            label_energy = output['particle_energy']
            
            particle_class = particle_class.squeeze().type(torch.FloatTensor).to(device)
            particle_energy = particle_energy.squeeze().type(torch.FloatTensor).to(device)
            
            
            y_pred_binary = label_class.cpu().detach().numpy()[:, 1]
            y_true_binary = particle_class.cpu().detach().numpy()
            y_pred_reg = label_energy.cpu().detach().numpy()
            y_true_reg = particle_energy.cpu().detach().numpy()
            
            y_pred_reg = [reverse_mapping[x] for x in list(np.argmax(y_pred_reg, axis = 1))]
            y_true_reg = [reverse_mapping[x] for x in list(y_true_reg)]
            comp_val_metric, roc_val, mae_val = calc_metric(y_true_binary, y_pred_binary, y_true_reg, y_pred_reg)
            
            
            # calculate loss
            loss_binary = criterion_binary(label_class, particle_class)
            loss_reg = criterion_reg(label_energy, particle_energy)
            
            loss = loss_binary + loss_reg
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
        
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tMetric: {:.6f} ROC AUC {:.6f} MAE {:.6f}'.format(
            epoch, train_loss, valid_loss, comp_val_metric, roc_val, mae_val))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            torch.save(model, 'model.pt')
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            valid_loss_min = valid_loss
    # return trained model
    return model

In [153]:
model_conv=train_model(model_CNN, criterion_binary, criterion_reg, optimizer)

tensor([1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1,
        1, 1, 0, 0, 0, 1, 1, 0], device='cuda:0') tensor([3, 2, 2, 3, 1, 1, 4, 5, 2, 1, 3, 2, 4, 5, 0, 5, 0, 4, 2, 1, 5, 3, 5, 5,
        5, 3, 2, 0, 4, 3, 1, 0], device='cuda:0')
Epoch 1, Batch 1 loss: 2.638103 Metric -16091.911765 ROC AUC 0.470588 MAE 16.562500
tensor([1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
        0, 0, 1, 1, 0, 0, 0, 1], device='cuda:0') tensor([5, 3, 0, 3, 4, 2, 4, 2, 1, 2, 3, 3, 1, 4, 1, 5, 3, 5, 1, 1, 3, 0, 5, 3,
        4, 0, 1, 1, 2, 2, 0, 3], device='cuda:0')
Epoch 1, Batch 2 loss: 2.606853 Metric -18086.538462 ROC AUC 0.538462 MAE 18.625000
tensor([0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
        0, 1, 0, 1, 0, 1, 1, 1], device='cuda:0') tensor([4, 3, 5, 1, 2, 3, 3, 3, 3, 5, 0, 4, 0, 1, 2, 1, 2, 3, 4, 4, 5, 4, 0, 0,
        4, 1, 2, 1, 2, 3, 5, 5], device='cuda:0')
Epoch 1, Batch 3 loss: 2.669353 Metric -16431.985294

KeyboardInterrupt: 